<!-- TABS -->
# Build image embedding model

...

In [ ]:
!wget https://raw.githubusercontent.com/openai/CLIP/main/CLIP.png

In [ ]:
image_path = "CLIP.png"

In [ ]:
# <tab: torchvision>
...

In [ ]:
# <tab: CLIP-multimodal>

# %pip install ipython opencv-python pillow openai-clip numpy==1.24.4

import clip
from superduperdb import vector
from superduperdb.ext.torch import TorchModel
from PIL import Image

# Load the CLIP model and define a tensor type
model, preprocess = clip.load("RN50")

# Create a TorchModel for visual encoding
model = TorchModel(
    identifier='clip_image',
    preprocess=preprocess,
    object=model.visual,
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
)

In [ ]:
# <tab: OpenAI>
...

In [ ]:
# <tab: Other>
...

In [ ]:
model.predict_one(Image.open(image_path))